In [ ]:
!pip install keras==2.7

In [2]:
!pip install tensorflow

In [1]:
from keras.preprocessing.image import ImageDataGenerator

In [2]:
import numpy as np
import tensorflow as tf
import PIL
from PIL import Image
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.layers import Conv2D,MaxPooling2D

In [3]:
train_datagen= ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip= True)
test_datagen= ImageDataGenerator(rescale=1./255)

In [4]:
pwd

'C:\\Users\\hariharan\\Downloads'

In [7]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='IxU5IONTCc_KneObVlEjH_rOao1GFxJwmYIKTJxVQwd8',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'arrhythmiaclassification-donotdelete-pr-t34dtwehaskeak'
object_key = 'Classification of Arrhythmia by Using Deep Learning with 2-D ECG Spectral Image Representation.zip'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


ModuleNotFoundError: No module named 'ibm_boto3'

In [8]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_paths= unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [9]:
import os
filenames=os.listdir('/home/wsuser/work/Classification of Arrhythmia by Using Deep Learning with 2-D ECG Spectral Image Representation/Dataset')
filenames

['train', 'test']

In [10]:
x_train=train_datagen.flow_from_directory(directory=r'/home/wsuser/work/Classification of Arrhythmia by Using Deep Learning with 2-D ECG Spectral Image Representation/Dataset/train', target_size=(64,64), batch_size=32, class_mode='categorical')
x_test=train_datagen.flow_from_directory(directory=r'/home/wsuser/work/Classification of Arrhythmia by Using Deep Learning with 2-D ECG Spectral Image Representation/Dataset/test', target_size=(64,64), batch_size=32, class_mode='categorical')

Found 15341 images belonging to 6 classes.
Found 6825 images belonging to 6 classes.


In [11]:
model=models.Sequential()

In [12]:
model.add(layers.Conv2D(32,(3,3), input_shape=(64, 64, 3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(32,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())

model.add(Dense(32))
model.add(Dense(6, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 32)                2

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(generator=x_train, steps_per_epoch=len(x_train), epochs=10, validation_data=x_test, validation_steps=len(x_test))

In [ ]:
model.save('ECG.h5')

In [ ]:
!tar -zcvf classification-model.tgz ECG.h5

In [ ]:
!pip install watson-machine-learning-client --upgrade

In [13]:
!pip install ibm-watson-machine-learning

In [14]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"LgbryrGJJuCoArJ91PEZMVqqjqMibx8kxnEEbAN3Wfxn"}
client=APIClient(wml_credentials)

In [15]:
def guid_from_space_name(client, space_name):
    space=client.spaces.get_details()
    #print(space)
    return(next(item for item in space ['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [16]:
space_uid= guid_from_space_name(client,'model_deployment')
print("Space UID -" + space_uid) 

Space UID -c8c36d59-c065-470d-b01d-c8543ebf1acf


In [17]:
client.set.default_space(space_uid)

'SUCCESS'

In [18]:
client.software_specifications.list(limit=100)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt

In [19]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [20]:
ls -1

 Volume in drive C is Acer
 Volume Serial Number is A2E3-5426

 Directory of C:\Users\hariharan\Downloads



File Not Found


In [21]:
model_details= client.repository.store_model(model='classification-model.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN" ,
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7" ,
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid})
model_id = client.repository.get_model_uid(model_details)

Failure during getting trained models details. (GET https://us-south.ml.cloud.ibm.com/ml/v4/trainings/classification-model.tgz?version=2021-06-24&space_id=c8c36d59-c065-470d-b01d-c8543ebf1acf)
Status code: 404, body: {"trace":"f3647461fde954b8e683f4b25b224baa","errors":[{"code":"training_job_run_not_found","message":"Backend persistence error (404): GET request failed","more_info":"http://watson-ml-api.mybluemix.net/"}],"status_code":"404"}
Unexpected type of 'model parameter', expected: model path / training_id, actual: 'classification-model.tgz'.


UnexpectedType: Unexpected type of 'model parameter', expected: model path / training_id, actual: 'classification-model.tgz'.

In [ ]:
model_id

In [ ]:
client.repository.download(model_id,'my_model.tar.gz' )

Testing the Model

In [ ]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
import tensorflow as tf
from tensorflow import keras
model = load_model("ECG.h5")



In [ ]:
x= image.load_img('/home/wsuser/work/Classification of Arrhythmia by Using Deep Learning with 2-D ECG Spectral Image Representation/Dataset/train/Ventricular Fibrillation/VFEfig_182.png',target_size=(64,64))
x= np.expand_dims(x, axis=0)
pred= model.predict(x)
pred

In [ ]:
index=['Left Bundle Branch Bloack','Normal','Premature Atrial Contraction','Premature Ventricular Contraction','Right Bundle Branch Block','Ventricular Fibrillation']
result=str(index[pred[0].tolist().index(1.)])
result